In [179]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import re
import morfeusz2
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.ticker as ticker
from pylab import cm
import matplotlib.font_manager as fm
mpl.rcParams['font.family'] = 'Avenir'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2
colors = cm.get_cmap('tab10', 2)
# sns.set(rc = {'figure.figsize':(20,10)})

In [2]:
morf = morfeusz2.Morfeusz()

In [3]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [5]:
data_desc = speech_data.copy()
data_desc["clean_text"] = data_desc["speech_text"].replace(to_replace = "(@\[0-9A-Za-ząćęłńóśżź,.():]+)|([^0-9A-Za-ząćęłńóśżź,.(): \t])|(\w+:\/\/\S+)|^rt|http.+?|\(.*\)"
                            ,regex=True
                            ,value="")

In [6]:
data_desc["text_lenght"] = data_desc["clean_text"].apply(lambda x: len(x))
data_desc["text_n_words"] = data_desc["clean_text"].apply(lambda x: len(x.split()))

In [224]:
n_speech = len(data_desc)
mean_text_len = np.mean(data_desc["text_lenght"])
std_text_len = np.std(data_desc["text_lenght"])
mean_text_n_words = np.mean(data_desc["text_n_words"])
std_text_n_words = np.std(data_desc["text_n_words"])
n_speech_per_speaker = data_desc["speaker"].value_counts()
mean_n_speech_per_speaker = np.mean(n_speech_per_speaker)
std_n_speech_per_speaker = np.std(n_speech_per_speaker)
n_speech_per_day = pd.DataFrame(data_desc.groupby("date").size(), columns = ["n_speech"]).reset_index()
mean_n_speech_per_day = np.mean(n_speech_per_day)
std_n_speech_per_day = np.std(n_speech_per_day)
n_sittings = len(pd.unique(data_desc["sitting"]))
n_days = len(pd.unique(data_desc["date"]))
n_speakers = len(pd.unique(data_desc["speaker"]))
n_speech_per_month = n_speech_per_day.set_index('date').groupby(pd.Grouper(freq='M'))['n_speech'].sum().reset_index()

E:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3417: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [286]:
speech_data_wo_dup = speech_data[~speech_data.last_name.isin(list(duplicates["last_name"]))].copy()

In [287]:
joined_data = pd.merge(speech_data_wo_dup, deputies[["last_name", "Party"]], how="inner", on="last_name")

In [289]:
data_desc_after_clean = joined_data.copy()

In [290]:
data_desc_after_clean["text_lenght"] = data_desc_after_clean["speech_text_only_author"].apply(lambda x: len(x))
data_desc_after_clean["text_n_words"] = data_desc_after_clean["speech_text_only_author"].apply(lambda x: len(x.split()))


In [291]:
n_speech_clean = len(data_desc_after_clean)
mean_text_len_clean = np.mean(data_desc_after_clean["text_lenght"])
std_text_len_clean = np.std(data_desc_after_clean["text_lenght"])
mean_text_n_words_clean = np.mean(data_desc_after_clean["text_n_words"])
std_text_n_words_clean = np.std(data_desc_after_clean["text_n_words"])
n_speech_per_speaker_clean = data_desc_after_clean["speaker"].value_counts()
mean_n_speech_per_speaker_clean = np.mean(n_speech_per_speaker_clean)
std_n_speech_per_speaker_clean = np.std(n_speech_per_speaker_clean)
n_speech_per_day_clean = pd.DataFrame(data_desc_after_clean.groupby("date").size(), columns = ["n_speech"]).reset_index()
mean_n_speech_per_day_clean = np.mean(n_speech_per_day_clean)
std_n_speech_per_day_clean = np.std(n_speech_per_day_clean)
n_sittings_clean = len(pd.unique(data_desc_after_clean["sitting"]))
n_days_clean = len(pd.unique(data_desc_after_clean["date"]))
n_speakers_clean = len(pd.unique(data_desc_after_clean["speaker"]))
n_speech_per_month_clean = n_speech_per_day_clean.set_index('date').groupby(pd.Grouper(freq='M'))['n_speech'].sum().reset_index()

E:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3417: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [162]:
mean_n_speech_per_day

n_speech    161.301724
dtype: float64

In [165]:
n_days

232

In [171]:
n_deputies = len(pd.unique(deputies["Name"]))
n_parties = len(pd.unique(deputies["Party"]))
n_dep_per_party = pd.DataFrame(deputies.groupby("Party").size(), columns = ["Name"]).reset_index().sort_values(by="Name", ascending=False)

In [174]:
n_dep_per_party

,Party,Name
7,PiS,238
3,PO-KO,148
5,PSL-KP,21
1,Kukiz15,18
11,niez.,11
9,UPR,4
0,Konfederacja,3
2,PO,3
4,PP,3
8,TERAZ!,3


In [ ]:
colors = sns.color_palette('pastel')[0:5]
fig = plt.figure(figsize=(15, 10))
ax = sns.barplot(x = "Party", y = "Name", data=n_dep_per_party)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
plt.xlabel("Ugrupowanie")
plt.ylabel("Liczba posłów")
fig.tight_layout()
plt.savefig("deputies_per_party.eps")
plt.show()

In [55]:
joined_data["mentions"] = joined_data["speech_text_only_author"].apply(lambda x: pd.unique([word[2][1].split(":")[0] for word in morf.analyse(x) if 'nazwisko' in word[2][3] and word[2][1].split(":")[0] in deputies_list]))

In [56]:
joined_data["mentioned_someone"] = joined_data["mentions"].apply(lambda x: len(x) != 0)
joined_data["mention_count"] = joined_data["mentioned_someone"].groupby(joined_data["speaker"]).transform("sum")
joined_data["mention_prob"] = joined_data["mentioned_someone"].groupby(joined_data["speaker"]).transform("sum") / joined_data["speech_text"].groupby(joined_data["speaker"]).transform("count")

In [57]:
mentioned_count = to_1D(joined_data["mentions"]).value_counts().to_frame().rename(columns={0: "mentioned_count"})
mentioned_count["last_name"] = to_1D(joined_data["mentions"]).value_counts().index
mentioned_count["be_mentioned_count"] = mentioned_count["mentioned_count"]
mentioned_count["be_mentioned_prob"] = mentioned_count["mentioned_count"] / len(joined_data)

In [78]:
deputies = pd.merge(deputies, mentioned_count[["last_name", "be_mentioned_count", "be_mentioned_prob"]], how="left", on="last_name")
deputies = pd.merge(deputies, joined_data[["last_name", "mention_count", "mention_prob"]], how="left", on="last_name").drop_duplicates().reset_index(drop=True).fillna(0)

In [118]:
duplicates = pd.merge(duplicates, mentioned_count[["last_name", "be_mentioned_count", "be_mentioned_prob"]], how="left", on="last_name")
duplicates = pd.merge(duplicates, joined_data[["last_name", "mention_count", "mention_prob"]], how="left", on="last_name").drop_duplicates().reset_index(drop=True).fillna(0)

In [87]:
# duplicates = duplicates[duplicates.duplicated(['last_name'], keep=False)]

In [137]:
joined_data

,sitting,day,date,number,title,speaker,speech_text,speech_text_only_author,last_name,Party,mentions,mentioned_someone,mention_count,mention_prob
0,54,4,2017-12-15,20.,Sprawozdanie Komisji Polityki Społecznej i Rod...,Abramowicz Adam,"Panie Marszałku! Wysoki Sejmie! Ci, którzy...","Panie Marszałku Wysoki Sejmie Ci, którzy mogą...",Abramowicz,PiS,[],False,26,0.666667
1,52,3,2017-11-24,12.,Sprawozdanie Komisji Polityki Społecznej i Rod...,Abramowicz Adam,Panie Marszałku! Wysoka Izbo! Sierpień 198...,"Panie Marszałku Wysoka Izbo Sierpień 1980 r.,...",Abramowicz,PiS,[Polak],True,26,0.666667
2,51,2,2017-09-11,19.,Pierwsze czytanie rządowego projektu ustawy o ...,Abramowicz Adam,Panie Marszałku! Wysoki Sejmie! System ube...,Panie Marszałku Wysoki Sejmie System ubezpiec...,Abramowicz,PiS,[],False,26,0.666667
3,51,2,2017-09-11,7.,Sprawozdanie Komisji Finansów Publicznych o rz...,Abramowicz Adam,Panie Marszałku! Wysoki Sejmie! Za czasów ...,Panie Marszałku Wysoki Sejmie Za czasów rządó...,Abramowicz,PiS,[],False,26,0.666667
4,48,2,2017-09-28,14.,Informacja bieżąca.,Abramowicz Adam,Panie Marszałku! Wysoki Sejmie! Panie Mini...,Panie Marszałku Wysoki Sejmie Panie Ministrze...,Abramowicz,PiS,"[Polak, Szydło]",True,26,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66110,5,1,2015-12-15,1.,Sprawozdanie Komisji Finansów Publicznych o rz...,Żyżyński Jerzy,Dziękuję bardzo. Ja w ramach niewykorzy...,Dziękuję bardzo. Ja w ramach niewykorzystaneg...,Żyżyński,PiS,[Kukiz],True,4,0.500000
66111,5,1,2015-12-15,2.,Sprawozdanie Komisji Finansów Publicznych o rz...,Żyżyński Jerzy,Panie Marszałku! Wysoka Izbo! Tutaj został...,Panie Marszałku Wysoka Izbo Tutaj zostały zap...,Żyżyński,PiS,[],False,4,0.500000
66112,4,1,2015-09-12,4. i 5.,4. Pierwsze czytanie rządowego projektu ustawy...,Żyżyński Jerzy,Witam. Panie Marszałku! Wysoka Izbo! Pa...,Witam. Panie Marszałku Wysoka Izbo Panie Mini...,Żyżyński,PiS,"[Polak, Skowrońska]",True,4,0.500000
66113,2,1,2015-11-25,2.,Pierwsze czytanie poselskiego projektu ustawy ...,Żyżyński Jerzy,Dziękuję. Panie Marszałku! Wysoka Izbo!...,Dziękuję. Panie Marszałku Wysoka Izbo Szanown...,Żyżyński,PiS,[],False,4,0.500000


In [132]:
duplicates

array(['Czarnecki Przemysław', 'Czarnecki Witold', 'Duda Antoni',
       'Duda Elżbieta', 'Duda Jan', 'Kamiński Mariusz', 'Kamiński Michał',
       'Kołakowski Lech', 'Kołakowski Robert', 'Krajewski Jarosław',
       'Krajewski Wiesław', 'Król Piotr', 'Król Wojciech',
       'Kubiak Dariusz', 'Kubiak Marta', 'Lipiec Grzegorz',
       'Lipiec Krzysztof', 'Maciejewski Andrzej', 'Maciejewski Krzysztof',
       'Małecki Jerzy', 'Małecki Maciej', 'Mucha Joanna', 'Mucha Rafał',
       'Nowak Maria', 'Nowak Tomasz Piotr', 'Nykiel Mirosława',
       'Nykiel Włodzimierz', 'Pawłowicz Krystyna', 'Pawłowicz Zbigniew',
       'Polak Marek', 'Polak Piotr', 'Sasin Jacek', 'Sasin Roman',
       'Schreiber Grzegorz', 'Schreiber Łukasz', 'Stawiarski Bartłomiej',
       'Stawiarski Jarosław', 'Suski Marek', 'Suski Paweł', 'Wilk Jacek',
       'Wilk Jerzy', 'Wilk Wojciech', 'Woźniak Grzegorz Adam',
       'Woźniak Tadeusz', 'Wójcik Marek', 'Wójcik Michał',
       'Wróblewska Kornelia', 'Wróblewska Krystyn

In [136]:
duplicates

,Name,Party,last_name
63,Czarnecki Przemysław,PiS,Czarnecki
64,Czarnecki Witold,PiS,Czarnecki
77,Duda Antoni,PiS,Duda
78,Duda Elżbieta,PiS,Duda
79,Duda Jan,PiS,Duda
159,Kamiński Mariusz,PiS,Kamiński
160,Kamiński Michał,PSL-KP,Kamiński
176,Kołakowski Lech,PiS,Kołakowski
177,Kołakowski Robert,PiS,Kołakowski
196,Krajewski Jarosław,PiS,Krajewski


In [ ]:
neig = {name: {n: 0 for n in deputies["last_name"].unique()} for name in deputies["last_name"].unique()}
for index, speech in joined_data.iterrows():
    for mention in speech["mentions"]:
        neig[speech["last_name"]][mention] += 1

In [ ]:
for name in neig.keys():
    neig[name] = dict(Counter(neig[name]))

In [ ]:
adj_matrix = pd.DataFrame(neig).T.fillna(0).astype(int)

In [ ]:
adj_matrix

In [ ]:
adj_matrix_diag = pd.Series(np.diag(adj_matrix), index=[adj_matrix.index, adj_matrix.columns]) 

In [ ]:
joined_data.iloc[36091]["speech_text"]

In [ ]:
adj_matrix_diag[adj_matrix_diag == 1]

In [ ]:
graph = nx.from_numpy_matrix(adj_matrix.values)

In [ ]:
d = dict(graph.degree)

In [ ]:
plt.figure(figsize=(30,20))
pos = nx.spring_layout(graph, k=2)
nx.draw(graph, pos , with_labels = True, width=0.4, 
        node_color='lightblue', node_size=[v * 100 for v in d.values()])

In [ ]:
G = nx.DiGraph()
for x, y in adj_matrix.iterrows():
    for yi, z in zip(y[y!=0].keys(), y[y!=0]):
        G.add_edge(x, yi, weight=z)

In [ ]:
degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
dmax = max(degree_sequence)

fig = plt.figure("Degree of a random graph", figsize=(8, 8))
# Create a gridspec for adding subplots of different sizes
axgrid = fig.add_gridspec(5, 4)

ax1 = fig.add_subplot(axgrid[3:, :2])
ax1.plot(degree_sequence, "b-", marker="o")
ax1.set_title("Degree Rank Plot")
ax1.set_ylabel("Degree")
ax1.set_xlabel("Rank")

ax2 = fig.add_subplot(axgrid[3:, 2:])
ax2.bar(*np.unique(degree_sequence, return_counts=True))
ax2.set_title("Degree histogram")
ax2.set_xlabel("Degree")
ax2.set_ylabel("# of Nodes")

fig.tight_layout()
plt.show()

In [ ]:
def degree_histogram_directed(G, in_degree=False, out_degree=False):
    """Return a list of the frequency of each degree value.

    Parameters
    ----------
    G : Networkx graph
       A graph
    in_degree : bool
    out_degree : bool

    Returns
    -------
    hist : list
       A list of frequencies of degrees.
       The degree values are the index in the list.

    Notes
    -----
    Note: the bins are width one, hence len(list) can be large
    (Order(number_of_edges))
    """
    nodes = G.nodes()
    if in_degree:
        in_degree = dict(G.in_degree())
        degseq=[in_degree.get(k,0) for k in nodes]
    elif out_degree:
        out_degree = dict(G.out_degree())
        degseq=[out_degree.get(k,0) for k in nodes]
    else:
        degseq=[v for k, v in G.degree()]
    dmax=max(degseq)+1
    freq= [ 0 for d in range(dmax) ]
    for d in degseq:
        freq[d] += 1
    return freq

In [ ]:
in_degree_freq = degree_histogram_directed(G, in_degree=True)
out_degree_freq = degree_histogram_directed(G, out_degree=True)
degrees = range(len(in_degree_freq))
plt.figure(figsize=(12, 8)) 
plt.loglog(range(len(in_degree_freq)), in_degree_freq, 'go-', label='in-degree') 
# plt.loglog(range(len(out_degree_freq)), out_degree_freq, 'bo-', label='out-degree')
plt.xlabel('Degree')
plt.ylabel('Frequency')